# Autoencoder Multivariate Training (Expiriment 3)

 Vinayak Sharma
  
 This notebook aims to train an autoencoder on univariate timeseries data from EKS Performance metrics for MANY NODES. The training data is 'non-anomalous'. 

 The model and implementation techniques can be found in the following github: https://github.com/emerelte/kad
 
 The research paper that this work is based on can be found here: https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9925210

------

In [2]:
##global variable
timesteps = 12
time_steps = 12
batch_size = 6
n_samples = batch_size*100
features = ['node_cpu_utilization','node_memory_utilization','node_network_total_bytes']

## Imports

In [3]:
# !pip install -r /root/eks-ml-pipeline/modeling/kad/kad/requirements.txt
# # # !pip install keras
# # # !pip install tensorflow

In [10]:
# pip install keras

In [11]:
# pip install keras

In [12]:
# import matplotlib as mpl
# mpl.rcParams.keys()

In [13]:
import importlib
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import json
import sys
import statsmodels.api as sm
import os
import keras
from keras import layers
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
# from pca_ad_dish_5g import pca_ad_dish_5g

from sklearn.model_selection import train_test_split

import _pickle as cPickle


In [14]:
import logging
import warnings
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
from matplotlib import pyplot as plt

In [15]:
pd.set_option('display.max_rows', None)

First lets read in our training sample set 

In [16]:
training_df_full = pd.read_parquet('/root/healthy_clusters_node_month.parquet')

In [17]:
training_df_full.head()

,Timestamp,InstanceId,ClusterName,node_cpu_utilization,node_cpu_limit,node_cpu_request,node_cpu_usage_total,node_memory_utilization,node_memory_request,node_memory_limit,node_network_rx_bytes,node_network_rx_dropped,node_network_rx_errors,node_network_rx_packets,node_network_total_bytes,node_network_tx_bytes,node_network_tx_dropped,node_network_tx_errors,node_network_tx_packets
0,1654235914008,i-01470d8d8e7b4fd2f,nk-ndc-eks-cluster-dev-usw2-az1-dev-ncm01,2.905276,4000.0,1266.0,116.211031,9.394841,998244352,16476487680,26133.384568,0.0,0.0,87.334264,56498.252063,30364.867495,0.0,0.0,86.300537
1,1654235917104,i-0b36e8825c482f762,nk-ndc-eks-cluster-dev-usw2-az1-dev-ncm01,1.836838,4000.0,1066.0,73.473529,7.296163,851443712,16476487680,5812.096097,0.0,0.0,22.781011,13940.610609,8128.514512,0.0,0.0,22.814785
2,1654232669947,i-01470d8d8e7b4fd2f,nk-ndc-eks-cluster-dev-usw2-az1-dev-ncm01,2.950336,4000.0,1266.0,118.013421,9.399117,998244352,16476487680,24538.169434,0.0,0.0,86.939803,54629.097665,30090.928231,0.0,0.0,86.904324
3,1654232681444,i-0b36e8825c482f762,nk-ndc-eks-cluster-dev-usw2-az1-dev-ncm01,1.858162,4000.0,1066.0,74.326468,7.295069,851443712,16476487680,5451.981961,0.0,0.0,21.206624,13392.905270,7940.923309,0.0,0.0,21.632175
4,1654235970093,i-093473861e74eaf2d,mt-ndc-eks-cluster-dev-mt-use1,0.942944,16000.0,1266.0,150.871041,4.956222,998244352,65902239744,42229.503040,0.0,0.0,251.934982,90291.404482,48061.901442,0.0,0.0,251.558398


In [18]:
columns_to_keep = ['Timestamp','InstanceId','node_cpu_utilization','node_memory_utilization','node_network_total_bytes']

In [19]:
training_df = training_df_full.drop(training_df_full.columns.difference(columns_to_keep),1, inplace=False)
training_df['Timestamp'] = pd.to_datetime(training_df['Timestamp'], unit='ms')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [20]:
len(training_df)

2636965

In [21]:
training_df = training_df.dropna()

In [22]:
test_df = training_df.copy()
training_df = training_df[training_df.InstanceId != 'i-0b36e8825c482f762']

-----

# TRAIN DATASET: Build out Dataset for training on many nodes

now we are starting to build out a training dataset for two nodes. This will prove our understanding of how to model can generalize across many EC2 instances. 

In [23]:
## for normalization
# scaler = StandardScaler()

scaler = StandardScaler()

In [24]:

instance_dfs =[]
for instance in training_df['InstanceId'].unique():
    instance_dfs.append(training_df[training_df.InstanceId == instance].sort_values(by='Timestamp')\
                        .reset_index(drop=True))

import random 

x_train = np.zeros((n_samples,time_steps,len(features)))
for b in range(n_samples):
    
    ##pick random df, and normalize
    df = random.choice(instance_dfs)
    df = df.drop(columns = ['InstanceId'])
    df = df.set_index('Timestamp')
    df = df.sort_index()
    df[features] = scaler.fit_transform(df[features])
    
    
    
    sample = np.zeros((n_samples,len(features)))
    ##make sure length of df is atleast 40
    first_time = random.choice(range(len(df)-time_steps))
    df.head()
    sample = df[features].iloc[first_time:first_time+time_steps]
    x_train[b] = sample

x_train.shape

(600, 12, 3)

our xtrain shape has the following properties: (sample size, time steps, numFeatures)

-------

# TEST DATASET: Build out Dataset for testing on one node

In [25]:
test_df = test_df[test_df.InstanceId == 'i-0b36e8825c482f762']

In [26]:
test_df.head()

,Timestamp,InstanceId,node_cpu_utilization,node_memory_utilization,node_network_total_bytes
1,2022-06-03 05:58:37.104,i-0b36e8825c482f762,1.836838,7.296163,13940.610609
3,2022-06-03 05:04:41.444,i-0b36e8825c482f762,1.858162,7.295069,13392.905270
9,2022-06-03 05:59:35.385,i-0b36e8825c482f762,1.824157,7.296387,14073.509120
11,2022-06-03 05:05:34.448,i-0b36e8825c482f762,1.821361,7.297928,14987.560707
17,2022-06-03 06:00:39.866,i-0b36e8825c482f762,1.811754,7.299718,13253.130119


In [27]:
## drop the instanceId
test_df = test_df.drop("InstanceId",1, inplace=False)


##set timestamp as the index
test_df = test_df.set_index('Timestamp')

##normalize test_df 
test_df[features] = scaler.fit_transform(test_df[features])


##ensure the data is sorted!!
test_df = test_df.sort_index()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


----

# Model Time

----

### First we train

In [28]:
import logging
import warnings
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from datetime import timedelta
from sklearn.preprocessing import StandardScaler

In [171]:
class pca_ad_dish_5g():
    """
    @:constructor takes in timesteps, batch size, learning rate, and a train_valid ratio
    
    @:returns object of class 
    """
    ## constructor takes in timesteps, batch size, learning rate, and a train_valid ratio
    ## timesteps is the number of time intervals inside of a sample
    ## batch size is number of  samples per iteration
    ## learning rate is the hyperparameter eta
    ## train_valid_ratio indicates the training and validation split crafted from the training set. IE the input dataset will be split for training and validation
    
    def __init__(self,num_of_features =3, number_of_temporal_slices = 6, timesteps_per_slice = 6, n_modes_to_delete=1):
        
        ## super().__init__() allows for inheritence amongst child classes
        super().__init__()
        
        ##define num_of_features
        self.num_of_features = num_of_features
            
        ##define number of temporal slices
        self.number_of_temporal_slices = number_of_temporal_slices
            
        ##set timesteps
        self.timesteps_per_slice = timesteps_per_slice
        
        self.N = number_of_temporal_slices * timesteps_per_slice
        
        ##create a results df
        self.results = None
        
        ##create an x_train
        self.x_train: np.ndarray = None
      
        ##create an x_train
        self.n_modes_to_delete= n_modes_to_delete
            
        ## CREATE the vectors for saving
        self.vs = None
        
        ##create encode and decode for 
        self.encode_decode_maps = None
        
        self.ss = None
        
    def two_time_slice(self, samples):
        rank4_sliced = samples.reshape(samples.shape[0],
                                       self.number_of_temporal_slices,
                                       self.timesteps_per_slice,
                                       self.num_of_features,
                                      )[:]
        
        rank3_sliced = rank4_sliced.reshape(samples.shape[0]*self.number_of_temporal_slices,
                                            self.timesteps_per_slice,
                                            self.num_of_features,
                                           )[:]
        return rank3_sliced, rank4_sliced

    def train(self, x_train):
        """
        @:param x_train: training data 
        Takes training set and:
            - fits the model
        @:returns nothing 
        """
        
        ##log that the autoencoder model training has begun
        logging.info("Autoencoder model training started")
        
        trainX_slices_as_samples, trainX_sliced  =  self.two_time_slice(x_train)
        
        
        self.ss = StandardScaler()
  
        # initializing, just for shape. The [:] is needed to have a copy instead of a view
        trainX_slices_as_samples_ss = trainX_slices_as_samples[:] 
        # initialize, since I have to do matmult by component
        trainX_slices_as_samples_ss_encoded = np.zeros(shape = (self.timesteps_per_slice - self.n_modes_to_delete ,trainX_slices_as_samples.shape[0],self.num_of_features ))
        trainX_slices_as_samples_ss_decoded = np.zeros(shape = (trainX_slices_as_samples.shape[0],self.timesteps_per_slice,self.num_of_features))

        # one feature at a time:
        for i in range(self.num_of_features):
            trainX_slices_as_samples_ss[:,:,i] = self.ss.fit_transform(trainX_slices_as_samples[:,:,i]) 

         
        # a list, one component for each feature, with Principal Vectors in a matrix V 
        vs = []
        pca = PCA(n_components = self.timesteps_per_slice)
        for i in range(self.num_of_features):
            pca.fit(trainX_slices_as_samples_ss[:,:,i]) # fits vectors across (n_samples, n_features)
            vs.append(pca.components_) # orthonormal basis of sample space in order of variance explained
         
        self.vs = vs
        
        ##create the encode decode maps
        self.encode_decode_maps = [np.matmul( vs[i][:-self.n_modes_to_delete,:].T, vs[i][:-self.n_modes_to_delete,:] ) for i in range(self.num_of_features) ]
        
        
        
        ##encode and decode using pca
        for i in range(self.num_of_features):
            trainX_slices_as_samples_ss_encoded[:,:,i] = np.matmul(vs[i][:-self.n_modes_to_delete,:], 
                                                          trainX_slices_as_samples_ss[:,:,i].T)
            trainX_slices_as_samples_ss_decoded[:,:,i] = np.matmul(vs[i][:-self.n_modes_to_delete,:].T, 
                                                          trainX_slices_as_samples_ss_encoded[:,:,i]).T
            
         
    
        # calculate residuals and errors
        residuals = trainX_slices_as_samples_ss - trainX_slices_as_samples_ss_decoded
        ed_errors = np.linalg.norm(residuals,
                    ord =1, # MAE, as in the rules
                    axis=1)
        
        return residuals,ed_errors,self.encode_decode_maps

    def test(self, x_test):
        """
        @:param x_test: test data. Must be of shape [ samples, timesteps, features]
        
        -apply inferencing
        
        @:returns decoding,residuals
        """
        testX_slices_as_samples, testX_sliced = self.two_time_slice(x_test)
        
        testX_slices_as_samples_ss = testX_slices_as_samples[:]
        
                # initialize, since I have to do matmul by feature
        testX_slices_as_samples_ss_encoded = np.zeros(shape = (self.timesteps_per_slice - self.n_modes_to_delete ,testX_slices_as_samples.shape[0],self.num_of_features ))
        testX_slices_as_samples_ss_decoded = np.zeros(shape = (testX_slices_as_samples.shape[0],self.timesteps_per_slice,self.num_of_features))
        
        ##apply standard scaler
        for i in range(self.num_of_features):
            testX_slices_as_samples_ss[:,:,i] = self.ss.fit_transform(testX_slices_as_samples[:,:,i])

            
       
    
        ##encode and decode using pca
        for i in range(self.num_of_features):
            testX_slices_as_samples_ss_encoded[:,:,i] = np.matmul(self.vs[i][:-self.n_modes_to_delete,:], 
                                                          testX_slices_as_samples_ss[:,:,i].T)
            testX_slices_as_samples_ss_decoded[:,:,i] = np.matmul(self.vs[i][:-self.n_modes_to_delete,:].T, 
                                                          testX_slices_as_samples_ss_encoded[:,:,i]).T
            
         
    
        
        
        residuals = testX_slices_as_samples_ss - testX_slices_as_samples_ss_decoded
        
        return testX_slices_as_samples_ss_decoded,residuals

In [172]:
pca_model = pca_ad_dish_5g(num_of_features=3, number_of_temporal_slices=3, timesteps_per_slice=4, n_modes_to_delete=1)

In [173]:
residuals,ed_errors,encode_decode_maps = pca_model.train(x_train)

In [174]:

residuals_reshaped = residuals.reshape(600,12,3)


### Then we test

In [180]:
y_hat_test = []
res_test = []

for i in range(0,12,timesteps):
    if(i + timesteps < len(test_df)):
        sample_topredict_on = test_df.iloc[i:i+timesteps]
        x_test = np.array(sample_topredict_on.to_numpy())
        x_test = x_test.reshape(1,-1,3)
        y_hat,residuals = pca_model.test(x_test)
        y_hat_reshape = y_hat.reshape(1,12,3) 
        res_reshape = residuals.reshape(1,12,3) 
        y_hat_test.append(y_hat_reshape)
        res_test.append(res_reshape)

#         print(y_hat.shape)
#         preds,errs,anom_scores = model.test(x_test)
        
#         predictions_f1.append(np.array(preds[:,:,0]))
#         predictions_f2.append(preds[:,:,1])
#         predictions_f3.append(preds[:,:,2])
        
#         anomaly_scoresf1.append(np.array(anom_scores[:timesteps,:]))
#         anomaly_scoresf2.append(anom_scores[timesteps:timesteps+timesteps,:])
#         anomaly_scoresf3.append(anom_scores[timesteps+timesteps:timesteps+timesteps+timesteps,:])
        
        
#         errors_f1.append(np.array(errs[:,:,0]))
#         errors_f2.append(errs[:,:,1])
#         errors_f3.append(errs[:,:,2])

### Visualize Testing

In [ ]:
1,12,3